In [1]:
import sys
from pathlib import Path
HOME = Path.cwd().parent
sys.path.append(str(HOME))

import pandas as pd

from tqdm import tqdm
from pathlib import Path
from collections import defaultdict

from src.data import BatteryData

In [2]:

DATA_HOME = Path('/workspace/battery/nmi_configs/data/processed')

all_materials = set()
ds_to_mat = {}

for data_source_path in DATA_HOME.glob('*'):
    datasource = data_source_path.stem
    ds_to_mat[datasource] = defaultdict(int)
    for battery in tqdm(list(data_source_path.glob('*.pkl')), desc=f'Processing {datasource}'):
        cell_data = BatteryData.load(battery)
        ds_to_mat[datasource][cell_data.cathode_material] += 1


Processing MATR:  22%|██▏       | 40/180 [00:15<00:36,  3.82it/s]Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f09ce558a30>>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 
Processing MATR:  49%|████▉     | 89/180 [00:26<00:22,  4.02it/s]

In [ ]:
ds_to_mat['HNEI'] = defaultdict(int)
ds_to_mat['HNEI']['NMC-LCO'] = 14
pd.DataFrame(ds_to_mat).fillna(0).astype(int).to_latex(
    '/workspace/battery/nmi_configs/nmi_rebuttal_final/assets/material_stats.tex'
)

Materials in organized datasets

In [1]:
import sys
import copy
import pickle
import numpy as np
import pandas as pd

from tqdm import tqdm
from pathlib import Path
from itertools import chain
from collections import defaultdict
from IPython.display import display

sys.path.append(str(Path.cwd().parent))
from src.data import DataBundle, BatteryData
from src.builders import TRAIN_TEST_SPLITTERS
from src.utils.config import import_config

HOME = Path.cwd().parent

In [2]:
%cd $HOME
dataset_folders = HOME / 'workspaces/baselines/sklearn/discharge_model/'

data_name_map = {
    'matr_1': 'MATR-1',
    'matr_2': 'MATR-2',
    'hust': 'HUST',
    'mix_20': 'MIX-20',
    'mix_100': 'MIX-100',
}

dataset_materials = {data: defaultdict(int) for data in data_name_map}

for dataset_folder in dataset_folders.glob('*'):
    dataset = data_name_map[dataset_folder.stem]
    config_file = next(dataset_folder.glob('config*.yaml'))
    config = import_config(config_file, ['train_test_split'])['train_test_split']
    train_cell_files, test_cell_files = TRAIN_TEST_SPLITTERS.build(config).split()

    train_cells = [
        BatteryData.load(c) for c in tqdm(
            train_cell_files, desc=f'Loading train cells for {dataset}'
        )
    ]
    test_cells = [
        BatteryData.load(c) for c in tqdm(
            test_cell_files, desc=f'Loading test cells for {dataset}'
        )
    ]
    for cell in chain(train_cells, test_cells):
        if 'HNEI' in cell.cell_id:
            cell.cathode_material = 'NMC-LCO'
        dataset_materials[dataset_folder.stem][cell.cathode_material] += 1

/workspace/battery/nmi_configs


Loading test cells for HUST: 100%|██████████| 22/22 [00:07<00:00,  3.11it/s]


In [5]:
df = pd.DataFrame(dataset_materials).fillna(0).astype(int)
df.columns = list(data_name_map.values())
df[['MATR-1', 'MATR-2', 'HUST', 'MIX-20', 'MIX-100']]

,MATR-1,MATR-2,HUST,MIX-20,MIX-100
LFP,83,81,77,275,257
LCO,0,0,0,23,23
NMC,0,0,0,69,48
NCA,0,0,0,22,0
NMC-LCO,0,0,0,14,14
